### Pivot

In [1]:
sales_data = [
    {"emp_id": 101, "name": "Arun",   "month": "Jan", "sales": 12000},
    {"emp_id": 101, "name": "Arun",   "month": "Feb", "sales": 15000},
    {"emp_id": 101, "name": "Arun",   "month": "Mar", "sales": 18000},
    {"emp_id": 102, "name": "Meena",  "month": "Jan", "sales": 10000},
    {"emp_id": 102, "name": "Meena",  "month": "Feb", "sales": 17000},
    {"emp_id": 102, "name": "Meena",  "month": "Mar", "sales": 16000},
    {"emp_id": 103, "name": "Kiran",  "month": "Jan", "sales": 9000},
    {"emp_id": 103, "name": "Kiran",  "month": "Feb", "sales": 11000},
    {"emp_id": 103, "name": "Kiran",  "month": "Mar", "sales": 13000},
    {"emp_id": 104, "name": "Divya",  "month": "Jan", "sales": 14000},
    {"emp_id": 104, "name": "Divya",  "month": "Feb", "sales": 16000},
    {"emp_id": 104, "name": "Divya",  "month": "Mar", "sales": 15000},
]

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

df = spark.createDataFrame(sales_data)
# df.show()


In [2]:
pivot_df = (
    df.groupBy("emp_id", "name")
      .pivot("month")
      .sum("sales")
)

pivot_df.show()


+------+-----+-----+-----+-----+
|emp_id| name|  Feb|  Jan|  Mar|
+------+-----+-----+-----+-----+
|   102|Meena|17000|10000|16000|
|   104|Divya|16000|14000|15000|
|   101| Arun|15000|12000|18000|
|   103|Kiran|11000| 9000|13000|
+------+-----+-----+-----+-----+



In [3]:
students_data = [
    {"student_id": 1, "name": "Arun",   "subject": "Math",    "marks": 88},
    {"student_id": 1, "name": "Arun",   "subject": "Science", "marks": 92},
    {"student_id": 1, "name": "Arun",   "subject": "English", "marks": 85},

    {"student_id": 2, "name": "Meena",  "subject": "Math",    "marks": 76},
    {"student_id": 2, "name": "Meena",  "subject": "Science", "marks": 81},
    {"student_id": 2, "name": "Meena",  "subject": "English", "marks": 78},

    {"student_id": 3, "name": "Kiran",  "subject": "Math",    "marks": 90},
    {"student_id": 3, "name": "Kiran",  "subject": "Science", "marks": 88},
    {"student_id": 3, "name": "Kiran",  "subject": "English", "marks": 84},

    {"student_id": 4, "name": "Divya",  "subject": "Math",    "marks": 69},
    {"student_id": 4, "name": "Divya",  "subject": "Science", "marks": 73},
    {"student_id": 4, "name": "Divya",  "subject": "English", "marks": 80},
]
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

df_s = spark.createDataFrame(students_data)
# df_s.show(5,truncate=False)

In [4]:
# Q1. Pivot the data to show each student's marks in Math, Science, and English in separate columns.
from pyspark.sql.functions import col,round,concat,lit
 
df_s = df_s.groupBy("student_id","name")\
    .pivot("subject")\
    .sum("marks")\
    .withColumn("total_marks",(col("English")+col("Math")+col("Science")))\
    .withColumn("Percentage", concat(round((col("total_marks")/300) * 100, 2),lit("%")))\
    .orderBy(col("total_marks").desc())

df_s.show(5)

+----------+-----+-------+----+-------+-----------+----------+
|student_id| name|English|Math|Science|total_marks|Percentage|
+----------+-----+-------+----+-------+-----------+----------+
|         1| Arun|     85|  88|     92|        265|    88.33%|
|         3|Kiran|     84|  90|     88|        262|    87.33%|
|         2|Meena|     78|  76|     81|        235|    78.33%|
|         4|Divya|     80|  69|     73|        222|     74.0%|
+----------+-----+-------+----+-------+-----------+----------+

